In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, date_format, unix_timestamp, from_unixtime, to_date, split, ltrim

import os
import shutil
import glob

In [10]:
spark = SparkSession.builder.appName('Challenge 1').getOrCreate()
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

DataFrame[key: string, value: string]

In [11]:
cons = spark.read.format("csv").option("header", "true").csv("./Exercise#1Data/cons.csv", inferSchema=True)
cons_email = spark.read.format("csv").option("header", "true").csv("./Exercise#1Data/cons_email.csv", inferSchema=True)
cons_email_chapter = spark.read.format("csv").option("header", "true").csv("./Exercise#1Data/cons_email_chapter_subscription.csv", inferSchema=True)

In [12]:
j1 = cons_email.alias("E").join(cons.alias('C'), col("E.cons_id") == col("C.cons_id"), 'left').selectExpr("E.email as email", "C.source as code", "C.create_dt as created_dt", "C.modified_dt as updated_dt", "E.cons_email_id as eid")
j2 = j1.alias('N').join(cons_email_chapter.alias('C'), col("N.eid") == col("C.cons_email_id"), 'left').where(col('C.chapter_id') == 1).selectExpr('N.*', 'C.isunsub as is_unsub')

In [13]:
def save_spark_df(df, file_path):
    temp_path = file_path + 'temp'
    df.coalesce(1).write.mode('overwrite').format('csv').option('header', 'true').save(temp_path)
    os.rename(glob.glob(temp_path + '/*.csv')[0], file_path)
    shutil.rmtree(temp_path)


In [14]:
j3 = j2.withColumn('is_unsub', col('is_unsub').cast('boolean'))
j3 = j3.withColumn('updated_dt', to_timestamp('updated_dt', 'EEE, yyyy-MM-dd HH:mm:ss'))
j3 = j3.withColumn('created_dt', to_timestamp('created_dt', 'EEE, yyyy-MM-dd HH:mm:ss'))
j3 = j3.drop('eid')

save_spark_df(j3, 'people.csv')

## Assumption

It is assumed that `created_dt` is the date the constituent was acquired.

In [15]:
people = spark.read.format("csv").option("header", "true").csv("./people.csv", inferSchema=True)

In [16]:
k = people.select(to_date('created_dt').alias('acquisition_date'))
k = k.groupby('acquisition_date').count().selectExpr('acquisition_date', 'count as acquisitions').sort(col('acquisition_date').desc())

save_spark_df(k, './acquisition_facts.csv')